In [1]:
%%capture

#@markdown Installing libs & files
!pip install mido
!pip install pretty_midi
!pip install mir_eval
!pip install note_seq==0.0.3
!pip install midi2audio
# !pip install gdown==4.2.0
!pip install --upgrade --no-cache-dir gdown
!wget https://musical-artifacts.com/artifacts/387/KBH_Real_and_Swell_Choir.sf2
!sudo apt-get install fluidsynth

''

In [ ]:
#@markdown install models


import gdown
url = "https://drive.google.com/file/d/1zjfrtq7-N8YKK2WE5iEfH-FpFghyUbir/view?usp=sharing"
gdown.download(url=url,  output="PIANO_MODELS.zip", fuzzy=True)

!unzip PIANO_MODELS.zip

#Setup

In [21]:
#@markdown Imports & Constants

import os
os.environ["KMP_SETTINGS"] = "false"

RNNTransformer_params = [
                      'embed_dim_',
                      'num_heads_',
                      'feed_forward_dim_',
                      'num_layers_enc_',
                      'key_dim_',
                      'HOP_LENGTH_', 
                      'auc_',
                      'ONLY_ONSET_',
                      'lstmU_',
                      ]

from math import *
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
print(tf.config.get_visible_devices())
from numpy.linalg import norm
import os
import gdown
import zipfile
import pickle
import os.path
from google.colab import files
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pathlib
import librosa
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import time
from librosa.filters import mel as mel_filter
from librosa.util import pad_center
from scipy.signal import get_window
import json
from abc import abstractmethod
from glob import glob
import soundfile
from torch.utils.data import Dataset
from tqdm import tqdm
# from tqdm.notebook import tqdm
import multiprocessing
import sys
import mido
import time
import pretty_midi
import collections
from mir_eval.util import hz_to_midi
import mir_eval
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from math import floor as floor_
from mir_eval.onset import evaluate as FPR_onset
from librosa import note_to_midi as N2M
from librosa import midi_to_note as M2N
import itertools
from midi2audio import FluidSynth
import note_seq
from note_seq import sequences_lib
from note_seq import constants
from note_seq import midi_io
from note_seq.midi_io import  note_sequence_to_midi_file, midi_to_note_sequence
from mido import Message, MidiFile, MidiTrack
from mir_eval.util import hz_to_midi, midi_to_hz
from math import ceil
from mir_eval.transcription import precision_recall_f1_overlap as FPR_tran
from mir_eval.onset import evaluate as FPR_onset
from midi2audio import FluidSynth
from IPython.display import Audio as PlayAudio
from IPython import display
import base64
import io
from scipy.io import wavfile
import pickle
from google.colab import output as output_
from note_seq import sequences_lib
from note_seq import constants
from note_seq.midi_io import  note_sequence_to_midi_file
from mido import Message, MidiFile, MidiTrack
from mir_eval.util import hz_to_midi

SAMPLE_RATE = 16000
MIN_MIDI = 21
MAX_MIDI = 108
MEL_FMIN = 30
WINDOW_LENGTH = 2048
HOP_LENGTH=512
ONLY_ONSET=False
ONLY_FRAME=False
CNNdeeper=False
Transformer_Model=False
EncoderLstm_Model = False
fs_path='KBH_Real_and_Swell_Choir.sf2'


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [25]:
#@markdown utils



def plot_sequences_midis_threhsold(MIDIs, threhsold_str='_0.4_'):
    for midi_ in MIDIs:
        if threhsold_str in midi_:
            print(midi_)
            sequence = note_seq.midi_file_to_note_sequence(midi_)
            note_seq.plot_sequence(sequence, )

def sinusoidal(shape, dtype = np.float32, min_scale: float = 1.0,
               max_scale: float = 10000.0,):
    '''
    sinusoidal positional embedding
    '''    
    if len(list(shape)) != 2:
        raise ValueError(
            f'Expected a 2D shape (max_len, features), but got {shape}.')
    max_len, features = shape
    output = np.zeros((max_len, features), dtype=dtype)
    position = np.arange(0, max_len)[:, np.newaxis]
    scale_factor = -np.log(max_scale / min_scale) / (features // 2 - 1)
    div_term = min_scale * np.exp(np.arange(0, features // 2) * scale_factor)
    output[:, :features // 2] = np.sin(position * div_term)
    output[:, features // 2:2 * (features // 2)] = np.cos(position * div_term)
    return output


def save_midi(path, pitches, intervals, velocities=None):
    '''
    save midi data as piano midi
    intervals: list -> each item[start_time, end_time]
    pitches: list(int)
    '''
    file = pretty_midi.PrettyMIDI()
    piano = pretty_midi.Instrument(program=0)
    for i, pitch in enumerate(pitches):
        interval=intervals[i]
        if velocities!=None:
            velocity = velocities[i]
        else:
            velocity=100
        note = pretty_midi.Note(velocity=velocity, pitch=pitch, start=interval[0], end=interval[1])
        piano.notes.append(note)


    file.instruments.append(piano)
    file.write(path)

def midi2wav(midi_path, wav_path, fs_path=fs_path, sr=16000):
    fs = FluidSynth(fs_path, sample_rate=sr)
    fs.midi_to_audio(midi_path, wav_path)


_play_count = 0 
def play_audio(audio_path, #array_of_floats,
         sample_rate=SAMPLE_RATE,
         ephemeral=True,
         autoplay=False,
         offset=10,
         duration=15):
  # DDSP play audio on google colab
  # https://github.com/magenta/ddsp/blob/main/ddsp/colab/colab_utils.py
  """Creates an HTML5 audio widget to play a sound in Colab.
  This function should only be called from a Colab notebook.
  Args:
    array_of_floats: A 1D or 2D array-like container of float sound samples.
      Values outside of the range [-1, 1] will be clipped.
    sample_rate: Sample rate in samples per second.
    ephemeral: If set to True, the widget will be ephemeral, and disappear on
      reload (and it won't be counted against realtime document size).
    autoplay: If True, automatically start playing the sound when the widget is
      rendered.
  """
  # If batched, take first element.
  array_of_floats, sr = librosa.load(audio_path, sr=sample_rate, duration=duration, offset=offset)
  if len(array_of_floats.shape) == 2:
    array_of_floats = array_of_floats[0]

  normalizer = float(np.iinfo(np.int16).max)
  array_of_ints = np.array(
      np.asarray(array_of_floats) * normalizer, dtype=np.int16)
  memfile = io.BytesIO()
  wavfile.write(memfile, sample_rate, array_of_ints)
  html = """<audio controls {autoplay}>
              <source controls src="data:audio/wav;base64,{base64_wavfile}"
              type="audio/wav" />
              Your browser does not support the audio element.
            </audio>"""
  html = html.format(
      autoplay='autoplay' if autoplay else '',
      base64_wavfile=base64.b64encode(memfile.getvalue()).decode('ascii'))
  memfile.close()
  global _play_count
  _play_count += 1
  if ephemeral:
    element = 'id_%s' % _play_count
    display.display(display.HTML('<div id="%s"> </div>' % element))
    js = output_._js_builder  # pylint:disable=protected-access
    js.Js('document', mode=js.EVAL).getElementById(element).innerHTML = html
  else:
    display.display(display.HTML(html))



In [3]:
#@markdown Mel spectrogram

def get_forward_basis(filter_length=2048, window='hann'):

    win_length = filter_length
    # window='hann' # blackmanharris hann
    fourier_basis = np.fft.fft(np.eye(filter_length))

    cutoff = int((filter_length / 2 + 1))
    fourier_basis = np.vstack([np.real(fourier_basis[:cutoff, :]),
                                np.imag(fourier_basis[:cutoff, :])])

    forward_basis = fourier_basis[:, None, :]
    forward_basis_c = forward_basis.copy()

    fft_window = get_window(window, win_length, fftbins=True, )
    fft_window = pad_center(fft_window, filter_length)
    forward_basis *= fft_window
    forward_basis = forward_basis.T.astype(np.float32)
    forward_basis = np.expand_dims(forward_basis, 0)
    forward_basis = tf.convert_to_tensor(forward_basis)

    return forward_basis

def get_mel_filter(sample_rate=16000, filter_length=2048, n_mels=229, mel_fmin=30, mel_fmax=8000, htk=True):

    mel_basis =  mel_filter(sample_rate, filter_length, n_mels, mel_fmin, mel_fmax, htk=htk)
    mel_basis = tf.convert_to_tensor(mel_basis)

    return mel_basis

class mel_tf(tf.keras.Model):
        
    def __init__(self,
                 filters,
                 mel_basis,
                 hop_size=512,
                 return_spec=False):
        super(mel_tf, self).__init__()
        self.filters = filters
        self.mel_basis = mel_basis
        self.hop_size = hop_size
        self.Dim1 =  filters.shape[1]
        self.Dim3 =  filters.shape[3]
        self.return_spec=return_spec
    def call(self, input_data):
        num_batches = input_data.shape[0]
        num_samples = input_data.shape[1]
        dim2 = int(num_samples/self.hop_size) + 1
        input_data = tf.pad(input_data, ([0, 0], [int(self.Dim1  / 2), int(self.Dim1  / 2)]), mode='REFLECT')
        input_data = tf.expand_dims(input_data, -1)
        input_data = tf.expand_dims(input_data, 1)
        forward_transform = tf.nn.conv2d(input_data, self.filters, self.hop_size, 'VALID')
        forward_transform = tf.squeeze(forward_transform)
        if num_batches > 1 :
            forward_transform = tf.transpose(forward_transform, perm=[0, 2, 1])
        else :
            forward_transform = tf.transpose(forward_transform)
            forward_transform = tf.reshape(forward_transform, [num_batches, self.Dim3, dim2])
        real_part = forward_transform[:, :int(self.Dim1  / 2 + 1), :]
        imag_part = forward_transform[:, int(self.Dim1  / 2 + 1):, :]
        magnitude = tf.math.sqrt(real_part**2 + imag_part**2)
        if self.return_spec:
            log_spec = tf.math.log(tf.clip_by_value(magnitude, clip_value_min =1e-5 , clip_value_max=np.inf))
            return log_spec
        else:
            mel_output = tf.matmul(self.mel_basis, magnitude)
            mel_output = tf.math.log(tf.clip_by_value(mel_output, clip_value_min =1e-5 , clip_value_max=np.inf))
            return mel_output



In [4]:
#@markdown Mt3_Encoder_Layer

class Mt3_Encoder_Layer(keras.layers.Layer):
    def __init__(self, embed_dim=512, num_heads=6,
                 feed_forward_dim=2048, key_dim=64,
                 rate=0.1):
        super().__init__()

        self.pre_attention_layer_norm = layers.LayerNormalization(epsilon=1e-6, name='pre_attention_layer_norm', scale=False)

        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=key_dim,  use_bias=False, name='attention'
        )

        self.pre_mlp_layer_norm = layers.LayerNormalization(epsilon=1e-6, name='pre_mlp_layer_norm', scale=False)

        self.mlp = keras.Sequential(
                    [
                        layers.Dense(feed_forward_dim, use_bias=False),#, activation='relu'
                        layers.Dense(embed_dim, use_bias=False),
                    ]
                )
    def set_norm_attention_norm_weights(self, weights):
        self.pre_attention_layer_norm.set_weights([weights[0]])
        self.attention.set_weights(weights[1:-1])
        self.pre_mlp_layer_norm.set_weights([weights[-1]])

    def freeze_norm_attention_norm(self):
        self.pre_attention_layer_norm.trainable=False
        self.attention.trainable=False
        self.pre_mlp_layer_norm.trainable=False

    def set_norm_attention_norm_trainable(self):
        self.pre_attention_layer_norm.trainable=True
        self.attention.trainable=True
        self.pre_mlp_layer_norm.trainable=True

    def call(self, input):

        x = self.pre_attention_layer_norm(input)
        x = self.attention(x, x)
        x = x + input
        y = self.pre_mlp_layer_norm(x)
        y = self.mlp(y)
        y= y + x

        return y


#@markdown Mt3_Encoder

class Mt3_Encoder(tf.keras.Model):
    def __init__(self, n_frames=200, embed_dim=512, num_heads=6,
                 feed_forward_dim=2048, key_dim=64,
                 n_layers=8, 
                 rate=0.1):
        super().__init__()
        self.layers_keys = ['layers_0', 'layers_1', 'layers_2', 'layers_3', 'layers_4', 'layers_5', 'layers_6', 'layers_7']

        self.continuous_inputs_projection=layers.Dense(embed_dim, use_bias=False)
        self.pos_encoding = sinusoidal((n_frames, embed_dim))
        self.mt3_encoder_layers = keras.Sequential(
            [
                Mt3_Encoder_Layer(embed_dim=embed_dim, num_heads=num_heads,
                                   feed_forward_dim=feed_forward_dim, key_dim=key_dim,
                                   rate=rate)
                for _ in range(n_layers)
            ]
        )
        self.encoder_norm = layers.LayerNormalization(epsilon=1e-6, name='encoder_norm', scale=False)

    def set_Encoder_weights_from_mt3(self, params):

        continuous_inputs_projection_weights = params['continuous_inputs_projection']['kernel']
        self.continuous_inputs_projection.set_weights([continuous_inputs_projection_weights])

        for ind, key in enumerate(self.layers_keys):
            layer_weigts = get_mt3_encoder_weights_to_Encoder_Norm_Attention_Norm(params[key])
            self.mt3_encoder_layers.layers[ind].set_norm_attention_norm_weights(layer_weigts)
            
    def freeze_copied_weights(self):
        self.continuous_inputs_projection.trainable=False
        for ind in range(len(self.layers_keys)):
            self.mt3_encoder_layers.layers[ind].freeze_norm_attention_norm()

    def set_copied_weights_trainable(self):
        self.continuous_inputs_projection.trainable=True
        for ind in range(len(self.layers_keys)):
            self.mt3_encoder_layers.layers[ind].set_norm_attention_norm_trainable()


    def call(self, x):
        
        x = self.continuous_inputs_projection(x)
        x = x + self.pos_encoding[tf.newaxis, :tf.shape(x)[1], :]
        x = self.mt3_encoder_layers(x)
        x = self.encoder_norm(x)

        return x






#@markdown EncoderLstm

class EncoderLstm(keras.Model):
    def __init__(
        self,
        encoder, 
        embed_dim=512,
        num_heads=6,
        feed_forward_dim=2048,
        key_dim=64,
        num_layers_enc=1,
        num_layers_dec=1,
        num_classes=2,
        batch_size=1,
        rate=0.0, 
        GaussianNoise=0.0,
        n_lstm_units=512,
        use_fc_instead=False,
    ):
        super().__init__()
        self.batch_size=batch_size
        self.num_layers_dec=num_layers_dec
        self.embed_dim=embed_dim
        self.num_classes=num_classes

        self.GaussianNoise_layer = tf.keras.layers.GaussianNoise(GaussianNoise)
        self.GaussianNoise = True if GaussianNoise>0 else False

        self.encoder = encoder

        self.use_fc_instead=use_fc_instead
        if use_fc_instead:
          self.decoder = layers.Dense(n_lstm_units, use_bias=False, activation='relu')
        else:
          self.decoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(n_lstm_units, return_sequences=True, return_state=True))

        self.classifier = layers.Dense(num_classes, use_bias=False, activation='sigmoid')

    def call(self, x):

        if self.GaussianNoise:
            x = self.GaussianNoise_layer(x)

        x = self.encoder(x)


        if self.use_fc_instead:
          y = self.decoder(x)
        else:
          y = self.decoder(x)[0]

        return self.classifier(y)



In [5]:
#@markdown Transcriber utils
n_rounds = None
Discard_len = None
Samples = None

DEBUG=True
def get_f0_pitches_from_frames(Frames_output, step_time=512/16000, threshold=0.5):

    Frames_output_thr = np.where(Frames_output > threshold, Frames_output, 0)
    pitches=[]
    f0=[]
    for frame_i, frame in enumerate(Frames_output_thr):
        if np.sum(frame) > 0:
            pidx = np.argmax(frame)

            pitch=pidx / 4
            if pitch%1==0.25 or pitch%1==0.75:
                pitch=round(pitch)
            elif pitch%1==0.5:
                if frame[pidx-1] < frame[pidx+1]:
                    pitch+=0.5
                else:
                    pitch-=0.5
                pitch=round(pitch)

            if DEBUG:
                print(f'frame_i {frame_i} pidx {pidx} {(pidx / 4)%1} {frame[pidx-1]} {frame[pidx]} {frame[pidx+1]} pitch {pitch}')

            f0.append([frame_i*step_time, midi_to_hz(pitch+MIN_MIDI)])
            pitches.append([frame_i*step_time, frame_i*step_time+step_time, int(pitch+MIN_MIDI)])
        else:
            f0.append([frame_i*step_time, 0])
            # pitches.append([frame_i*step_time, frame_i*step_time+step_time, 0])

    return np.asarray(pitches), np.asarray(f0)

def get_pitches_from_frames(Frames_output_thr, step_time=512/16000):
    pitches=[]
    for frame_i, frame in enumerate(Frames_output_thr):
        if np.sum(frame) > 0:
            pitches_indexes = np.where(frame==True)[0]
            for pitch in pitches_indexes:
              pitches.append([frame_i*step_time, frame_i*step_time+step_time, pitch])
    return np.asarray(pitches)

def get_Values(n, Input_len, Overlap_len):
    # n = 3300
    # Input_len = 13
    # Overlap_len = 7
    global n_rounds
    global Discard_len
    global Samples
    Samples = np.arange(n)

    Discard_len = Input_len - Overlap_len

    n_rounds = int((n-Overlap_len)/Discard_len)
    Ratio = Input_len/Discard_len
    # print(n_rounds, Ratio)

    Inputs = []
    for i in range(n_rounds):
        from_ = i*Discard_len
        to_ = from_ + Input_len
        input_ = Samples[from_:to_]
        Inputs.append(list(input_))

    Inputs = np.array(Inputs)# first and last
    # print(Inputs.shape)

    uni, counts = np.unique(Inputs, return_counts=True)
    importand_counts = counts[Overlap_len:-Overlap_len]
    cumsum_ = np.cumsum(importand_counts)
    index_ = list(cumsum_).index(Input_len) + 1
    assert (importand_counts[:index_]==importand_counts[index_:index_*2]).all()
    assert (importand_counts[:index_]==importand_counts[index_:index_*2]).any()
    # print(importand_counts[:index_],'\n\n')

    # print(np.unique(importand_counts[:index_], return_counts=True), np.sum(importand_counts[:index_]),'\n\n')

    for i in range(1, 5):
        check_ = i*Input_len
        index_check = list(cumsum_).index(check_)+1
        assert index_check/(index_)%1==0
        # print(index_check, index_check/(index_))

    # print(counts[:Overlap_len])
    Values = 1/importand_counts[:index_]
    return Values

def get_pitches_intervels_orff_output_for_onset_output(Frames, Accepted_Midis_Values, step_time, connect_ms_threshold=200):

    connect_s_threshold = connect_ms_threshold/1000
    Pitches_dic = {}

    for frame_i, frame in enumerate(Frames):
        for pitch_indx in np.where(frame>0)[0]:
            pitch = Accepted_Midis_Values[pitch_indx]
            if not pitch in Pitches_dic:
                Pitches_dic[pitch]=[]
            Pitches_dic[pitch].append(frame_i*step_time)

    Pitches = []
    Intervels = []

    for pitch in Pitches_dic:
        onsets = Pitches_dic[pitch]
        if len(onsets)>1:
            Intervel=[]
            last_onset = onsets[0]
            last_offset = last_onset+step_time
            for onset in onsets[1:]:
                if (onset - last_offset)<=connect_s_threshold:
                    last_offset = onset +  step_time
                else:
                    Pitches.append(pitch)
                    Intervels.append([last_onset, last_offset])
                    last_onset = onset
                    last_offset = last_onset + step_time
            Pitches.append(pitch)
            Intervels.append([last_onset, last_offset])
        else:
            Pitches.append(pitch)
            Intervels.append([onsets[0], onsets[0]+step_time])


    return Pitches, Intervels


In [6]:
#@markdown Transcriber

class Transcribe:

    def __init__(self, model,
                 length_ms,
                 hop_sizes = [512],
                jumping_32mss = 5,
                thresholds = [0.5],
                connected_range_ms = 160,
                ONLY_FRAME=False,
                ONLY_ONSET=False,
                frequency_labels=False,
                connect_ms_threshold=0,
                 ):
        '''
        length_ms : length in milliseconds
        overlap from left and right in milliseconds
        connected_range_ms in milliseconds
        '''
        self.hop_sizes = hop_sizes
        self.length_ms = length_ms
        self.length_samples = int(self.length_ms * 0.001 * SAMPLE_RATE )
        self.jumping_samples = jumping_32mss * 512

        self.thresholds = thresholds
        self.connected_range_ms = connected_range_ms

        self.model = model
        self.model.sequence_length=self.length_samples
        self.model.batch_size=1
        self.MIDIs = []
        self.ONLY_FRAME=ONLY_FRAME
        self.ONLY_ONSET=ONLY_ONSET
        self.frequency_labels=frequency_labels
        self.connect_ms_threshold=connect_ms_threshold


    def Convert_output2Midi(self, MODEL_OUTPUT, hop_size):

        if self.frequency_labels:
            Frames_output=np.zeros((MODEL_OUTPUT.shape[0], 88))
            index_c=-3
            weights=[0.0, 0.25, 0.75, 3., 0.75, 0.25, 0.0]
            weights=np.asarray(weights)/2
            weights=list(weights)
            for k in range(88):
                temp_indices=[]
                temp_weights=[]
                for w in range(7):
                    if index_c>=0 and index_c<88*7:
                        # print("w", w, index_c, temp_indices)
                        temp_indices.append(index_c)
                        temp_weights.append(weights[w])
                    index_c+=1
                # index_c-=1
                index_c-=3
                # print(index_c, temp_indices)
                Frames_output[:,k]=(MODEL_OUTPUT[:,temp_indices]*temp_weights).sum(1)

            for i, v in enumerate(Frames_output.max(1)):
                Frames_output[i]=(Frames_output[i]==v).astype(float)*v
        elif self.ONLY_FRAME or self.ONLY_ONSET:#TODO new
            Frames_output = MODEL_OUTPUT
        else :
            Onsets_output = MODEL_OUTPUT[:,:88]
            Frames_output = MODEL_OUTPUT[:,88:88*2]
            OFFSETS_output = MODEL_OUTPUT[:,88*2:]


        for threshold in self.thresholds:
            if self.ONLY_FRAME or self.ONLY_ONSET:#TODO new
                Frames_t = np.where(Frames_output>threshold, True, False)
                midi_path = self.midi_base_name+"_"+str(threshold)+"_"+str(hop_size)+"_"+str(self.length_ms)+".mid"
                DEFAULT_FRAMES_PER_SECOND = SAMPLE_RATE / hop_size
                sequence = sequences_lib.pianoroll_to_note_sequence(
                    Frames_t, frames_per_second=DEFAULT_FRAMES_PER_SECOND,
                        min_duration_ms=0, min_midi_pitch=0,  )

            else :
                f_threshold = threshold/2
                Onsets_t = np.where(Onsets_output>threshold, Onsets_output, 0)
                Frames_t = np.where(Frames_output>f_threshold, True, False)
                OFFSETS_t = np.where(OFFSETS_output>threshold, OFFSETS_output, 0)
                midi_path = self.midi_base_name+"_"+str(threshold)+"_"+str(hop_size)+"_"+str(self.length_ms)+".mid"
                DEFAULT_FRAMES_PER_SECOND = SAMPLE_RATE / hop_size
                sequence = sequences_lib.pianoroll_to_note_sequence(
                    Frames_t,  onset_predictions=Onsets_t, offset_predictions=OFFSETS_t,
                    frames_per_second=DEFAULT_FRAMES_PER_SECOND, min_duration_ms=0, min_midi_pitch=0, )

            print("For threshold {} -- Number of notes {}".format(threshold, len(sequence.notes)))

            midi_arr = np.zeros((len(sequence.notes), 3))
            for i in range(len(sequence.notes)):
                midi_arr[i][0] = sequence.notes[i].start_time + self.offset_time
                midi_arr[i][1] = sequence.notes[i].end_time + self.offset_time
                midi_arr[i][2] = sequence.notes[i].pitch+1


            try :
                # TODO save_midi
                save_midi(midi_path, midi_arr[:,2].astype(int)+MIN_MIDI-1, midi_arr[:,:2])
                self.MIDIs.append(midi_path)
                if self.ONLY_ONSET and False :
                    midi_arr_onsets = get_pitches_from_frames(Frames_t)
                    midi_path_2 = midi_path.replace('.mid', '_onsets.mid')
                    save_midi(midi_path_2, midi_arr_onsets[:,2].astype(int)+MIN_MIDI, midi_arr_onsets[:,:2])
                    self.MIDIs.append(midi_path_2)
            except :
                if len(midi_arr) == 0:
                    print("Empty")
                else:
                    print("Error ")
                    print(midi_arr)

        return sequence

    def call(self, audio_path,
             audio_length=None,
             random_input_samples = None,
             midi_base_name = 'test', audio_from=None, audio_to=None):
        '''
        audio_length : length in milliseconds
        '''
        # TODO : length_samples must be a multiple of 512
        # TODO : audio and audio length must be a multiple of 512
        self.midi_base_name=midi_base_name
        audio, sr = librosa.load(audio_path, sr=SAMPLE_RATE)
        print("\nAudio {} shape {} SampleRate {}".format(audio_path, audio.shape, sr))
        if len(audio.shape) == 2:
            shape_len = len(audio.shape)
            audio = np.sum(audio, axis=1)/shape_len

        self.offset_time = 0.0
        if audio_from or audio_to:
            # from_ = np.random.randint(audio.shape[0] - sequence_length)
            self.offset_time = audio_from
            audio = audio[int(audio_from*SAMPLE_RATE):int(audio_to*SAMPLE_RATE)]
        if random_input_samples :
            audio = np.random.random((random_input_samples))
        if audio_length :
            #TODO
            sequence_length = int(audio_length * 0.001 * SAMPLE_RATE )
            # from_ = np.random.randint(audio.shape[0] - sequence_length)
            audio = audio[:sequence_length]

        # n_pad = self.jumping_samples - (audio.shape[0] - self.length_samples)%self.jumping_samples
        # if n_pad < self.jumping_samples :
        #     audio = np.pad(audio, (0, n_pad), 'constant')

        for hop_size in self.hop_sizes :
            self.step_time=hop_size/16000
            length_bins = int(self.length_samples / hop_size)
            jumping_bins = int(self.jumping_samples / hop_size)
            mel_model=mel_tf_model
            tic_ = time.time()
            connected_range_bins = int((self.connected_range_ms * 0.001 * SAMPLE_RATE ) / hop_size)
            range__ = (audio.shape[0] - self.length_samples) //  self.jumping_samples + 1
            notes_threshold_dic = {}
            model_outputs = []
            if self.ONLY_FRAME  or self.ONLY_ONSET:
                MODEL_OUTPUT = np.zeros((int(self.length_samples/hop_size) + int(range__*(self.jumping_samples/hop_size)), 88))
            else :
                MODEL_OUTPUT = np.zeros((int(self.length_samples/hop_size) + int(range__*(self.jumping_samples/hop_size)), 264))

            for i, from_i in enumerate(range(range__)):
                print("\r", from_i, '/', range__, end='')
                from_ = from_i * self.jumping_samples
                to_ = from_ + self.length_samples
                input_ = np.expand_dims(audio[from_:to_ - 1], 0)
                # print("input_", input_.shape)
                mel_output = mel_model(input_)
                mel_output = tf.transpose(mel_output, perm=[0, 2, 1])
                # print("mel_output", mel_output.shape)

                # mel_output = tf.reshape(mel_output, [mel_output.shape[0], mel_output.shape[1], mel_output.shape[2]])#TODO
                # print(MODEL_OUTPUT.shape, mel_output.shape)
                model_output = self.model(mel_output).numpy()[0] # TODO : batch has to be 1 if more we need to loop
                # print(mel_output.shape)


                if i == 0 :
                    # print(MODEL_OUTPUT.shape, model_output.shape)
                    MODEL_OUTPUT[:ceil(self.length_samples/hop_size)] = model_output
                else :
                    temp_n_1 = ceil(self.length_samples/hop_size)
                    temp_n = ceil((self.jumping_samples/hop_size))
                    # print(temp_n_1, temp_n, self.jumping_samples)
                    MODEL_OUTPUT[temp_n*i:temp_n*(i-1)+temp_n_1] = (MODEL_OUTPUT[temp_n*i:temp_n*(i-1)+temp_n_1] + model_output[:-temp_n])/1
                    MODEL_OUTPUT[temp_n*(i-1)+temp_n_1:temp_n*(i)+temp_n_1] = model_output[-temp_n:]

            Input_bins = self.length_samples/hop_size
            overlap_bins = Input_bins - self.jumping_samples/hop_size
            assert Input_bins%1==0
            assert overlap_bins%1==0
            Input_len=int(Input_bins)
            Overlap_len=int(overlap_bins)
            if Overlap_len>0 and Overlap_len<Input_len:
                Values = get_Values(n=Input_len*10, Input_len=Input_len, Overlap_len=Overlap_len)
                n_Values = Values.shape[0]
                indecies_values = (((np.arange(MODEL_OUTPUT.shape[0]) - Overlap_len) / n_Values) % 1 ) * n_Values
                # MODEL_OUTPUT[:-Input_len] *= Values[indecies_values.astype(int)][:-Input_len,np.newaxis]
                MODEL_OUTPUT *= Values[indecies_values.astype(int)][:,np.newaxis]
                print(f'Input_bins {Input_len} overlap_bins {Overlap_len} Values {Values}')

            print("\r", end='')
            toc_ = time.time()
            Tran_time=toc_ - tic_
            print("Transcription Time:{} Inference time per-round:{}".format(Tran_time, round((Tran_time/range__)*1000, 3) ))
            sequence = self.Convert_output2Midi(MODEL_OUTPUT, hop_size)


        return MODEL_OUTPUT, sequence, self.MIDIs


#Choose model and load it

In [7]:
#@markdown  Choose model


def update_model_params(folder_name):
    if folder_name.startswith('EncoderLstm'):
        global EncoderLstm_Model
        EncoderLstm_Model=True
        global ONLY_ONSET
        ONLY_ONSET =True
        for param in RNNTransformer_params:
            try:
                val = folder_name.split(param)[1]
                indx = val.find('_')
                try:
                    val = int(val[:indx])
                except:
                    val = val[:indx]
                exec(f'''global {param}\n{param}={val}''')
            except:
                pass


urls = [
    'https://drive.google.com/drive/folders/1LxDf0mfpaTR5X4VwhXcwubRYx36VK2pP?usp=sharing',
 'https://drive.google.com/drive/folders/1QrPmHU9HNb_OpAQSwZVExw26xa4tXHNj?usp=sharing',
   'https://drive.google.com/drive/folders/1vzk4DA0pQCXAx7pzdfrmPCTt88BAq5cI?usp=sharing',
     'https://drive.google.com/drive/folders/1hkoRwkOUItUe5swGMCRTq9bboIBHi5yP?usp=sharing',
]


Model_names = [
    'EncoderLstm_embed_dim_256_num_heads_6_feed_forward_dim_2048_num_layers_enc_8_key_dim_64_sequence_length_32768_HOP_LENGTH_512_lstmU_512__orgCho12__auc_87_130441',
    'EncoderLstm_embed_dim_256_num_heads_6_feed_forward_dim_2048_num_layers_enc_8_key_dim_64_sequence_length_16384_HOP_LENGTH_512_lstmU_512__OrgCho12Cent__auc_88_136149',
    'EncoderLstm_embed_dim_256_num_heads_6_feed_forward_dim_2048_num_layers_enc_8_key_dim_64_sequence_length_16384_HOP_LENGTH_512_lstmU_512_OrgCho12Aug__auc_85_142105',
    'EncoderLstm_embed_dim_500_num_heads_6_feed_forward_dim_2048_num_layers_enc_8_key_dim_64_sequence_length_16384_HOP_LENGTH_512_lstmU_512_OrgCho12Aug__auc_79_157789',
]

choice = "Piano 1 - Only Onset" #@param ['Piano 1 - Only Onset', 'Piano 2 - Only Onset', 'Piano 3 - Only Onset', 'Piano 4 - Only Onset']


print(f'choice {choice}')
choices = ['Piano 1 - Only Onset', 'Piano 2 - Only Onset', 'Piano 3 - Only Onset', 'Piano 4 - Only Onset']

choice = choices.index(choice)

print(choice, choices[choice])

print(f'choice {choice} urls {len(urls)}')

url = urls[choice]
Model_Name = Model_names[choice]
if not os.path.isdir(Model_Name):
    !gdown --folder $url

print(f'Model URL :    {url}')

print(Model_Name)
Model_weights_path=Model_Name+'/keras_model'

update_model_params(Model_Name)



choice Piano 1 - Only Onset
0 Piano 1 - Only Onset
choice 0 urls 4
Model URL :    https://drive.google.com/drive/folders/1LxDf0mfpaTR5X4VwhXcwubRYx36VK2pP?usp=sharing
EncoderLstm_embed_dim_256_num_heads_6_feed_forward_dim_2048_num_layers_enc_8_key_dim_64_sequence_length_32768_HOP_LENGTH_512_lstmU_512__orgCho12__auc_87_130441


In [11]:
#@markdown load model



if EncoderLstm_Model==True:
    num_classes=88
    global ONLY_ONSET
    ONLY_ONSET=True
    embed_dim=embed_dim_
    num_heads=num_heads_
    feed_forward_dim=feed_forward_dim_
    key_dim=key_dim_
    num_layers_enc=num_layers_enc_
    num_classes=num_classes
    n_lstm_units=lstmU_
    mel_bins = embed_dim
    batch_size = 1
    
    sequence_length = 50*512
    n_frames = sequence_length//HOP_LENGTH

    encoder=Mt3_Encoder(n_frames=200, 
                            embed_dim=embed_dim, 
                            num_heads=num_heads,
                            feed_forward_dim=feed_forward_dim,
                            n_layers=num_layers_enc,
                            key_dim=key_dim,
                            rate=0.0,
                            )
    
    try:
      encoder(np.ones((1, n_frames, mel_bins))).shape
    except:
      encoder(np.ones((1, n_frames, 256))).shape


    filters = get_forward_basis(window='hann', filter_length=WINDOW_LENGTH)
    mel_basis = get_mel_filter(n_mels=mel_bins, filter_length=WINDOW_LENGTH)

    print(f'filters {filters.shape}')

    EncoderFC = True if 'EncoderFC' in Model_weights_path else False
    model = EncoderLstm(
                encoder=encoder,
                batch_size=batch_size,
                embed_dim=embed_dim,
                num_heads=num_heads,
                feed_forward_dim=feed_forward_dim,
                key_dim=key_dim,
                num_layers_enc=num_layers_enc,
                num_classes=88,
                n_lstm_units=n_lstm_units,
                rate=0,
                GaussianNoise=0,
                use_fc_instead=EncoderFC
                )

    dump_input=np.random.random((batch_size, int(sequence_length//HOP_LENGTH), mel_bins))
    print('output',  model(dump_input).shape)

    model.load_weights(Model_weights_path)
    print("Layer one sum ", np.sum(model.weights[0]))
    # print(model.summary())
    model(dump_input).shape

    print(f"Model %s Is Ready"%(Model_weights_path.split('/')[-2][-10:]))

filters (1, 2048, 1, 2050)
output (1, 50, 88)
Layer one sum  -14.900374
Model _87_130441 Is Ready


#Transcribe Audios

In [14]:
#@title Upload
uploaded = files.upload()
Files = []
for name in uploaded.keys():
    Files.append(name)

Saving 599-36-eval-1.wav to 599-36-eval-1.wav


In [15]:
Files=glob('*.wav')
Files

['599_36_fast.wav', '599-36-eval-1.wav']

### Transcribe normally

In [16]:
#@title Parameters
thresholds = [0.35, 0.25, 0.3, 0.1, 0.5]#@param
length_ms = 1024 # input to model in each iteration in milliseconds #@param

hop_sizes= [HOP_LENGTH] #@ param
audio_from= None # starting second#@param
audio_to= None # ending second#@param
overlap_ms = 512 # value should be a multiple of 32 in milliseconds #@param
if overlap_ms >= length_ms:
    print("overlap_ms should be smaller than length_ms")
    raise "overlap_ms should be smaller than length_ms"

jumping_32mss= length_ms//32 - overlap_ms//32

connect_ms_threshold=-1
mel_tf_model = mel_tf(filters,
                        mel_basis,
                        HOP_LENGTH)
  
dim2 = int(length_ms * 0.001 * SAMPLE_RATE )
dim2_ = ceil(dim2/HOP_LENGTH)
sinusoidal_t = tf.convert_to_tensor(sinusoidal((dim2_, mel_bins)))


print(sinusoidal_t.shape, hop_sizes, jumping_32mss)

(32, 256) [512] 16


In [18]:
#@title Transcribe

tic = time.process_time()
MIDIs = []

for i_file, file_ in enumerate(Files) :
    print("\nFile Number {} / {}".format(i_file+1, len(Files)))

    audio_path = file_
    Midi_to_test_path = ''
    Midi_base_name = Midi_to_test_path+audio_path.split('.')[0]
    Transcribe_model = Transcribe(model, length_ms=length_ms, hop_sizes= hop_sizes,
                            jumping_32mss=jumping_32mss, thresholds=thresholds,
                            ONLY_ONSET=ONLY_ONSET,
                                frequency_labels=False,
                                  connect_ms_threshold=connect_ms_threshold,
                                  )

    MODEL_OUTPUT, sequence, t_MIDIs = Transcribe_model.call(audio_path=audio_path,
                                                        midi_base_name=Midi_base_name,
                                        audio_from=audio_from ,audio_to=audio_to)
    for i in t_MIDIs:
        MIDIs.append(i)

    toc = time.process_time()

print("\nTime: ", toc - tic)

 36 / 37Input_bins 32 overlap_bins 16 Values [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
Transcription Time:11.060771226882935 Inference time per-round:298.94
For threshold 0.35 -- Number of notes 86
For threshold 0.25 -- Number of notes 132
For threshold 0.3 -- Number of notes 104
For threshold 0.1 -- Number of notes 605
For threshold 0.5 -- Number of notes 50

Time:  25.832748353


In [ ]:
plot_midis_for_threshold='0.35'#@param
plot_sequences_midis_threhsold(MIDIs, threhsold_str=f'_{plot_midis_for_threshold}_')

In [ ]:
#@markdown listen to midi
convert_midi2wav_with_threshold='0.1'#@param
threhsold_str = f'_{convert_midi2wav_with_threshold}_'
for midi_ in MIDIs:
    if threhsold_str in midi_:
        midi2wav(midi_, midi_.replace('.mid', '.wav'), fs_path=fs_path)
        play_audio(midi_.replace('.mid', '.wav'))

In [75]:
#@title Download MIDIs
from zipfile import ZipFile

file_name = "MIDI_FILES.zip"

with ZipFile(file_name,'w') as zip:
        for midi_ in MIDIs:
            zip.write(midi_)

files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# !ffmpeg -i m.mp4 m.wav